In [ ]:
import os
os.environ['JAVA_HOME'] = '/home/aiops/zhuty/jdk-11.0.20'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']


In [ ]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader

index_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_20b_id_added/bm25_index"
index_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/bm25_index"
index_reader = IndexReader(index_path)
index_reader.stats()

In [ ]:
doc_vector = index_reader.get_document_vector('0_0')
print(doc_vector)

In [ ]:
searcher = LuceneSearcher(index_path)
doc = searcher.doc('0_0')
print(doc.raw())
print(doc.contents())

In [ ]:
doc.contents()

In [ ]:
doc.lucene_document(), doc.raw()


In [ ]:
hits = searcher.search('What is the best way to prepare for a job interview?', k=100)
for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

In [ ]:
import os

In [ ]:
import json
docid = "3_10026"
chunk_id, seq_id = docid.split("_")
base_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/train"
base_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/queries"
jsonl_file = os.path.join(base_path, "chunk_{}.jsonl".format(chunk_id))
with open(jsonl_file, "r") as f:
    for line in f:
        line = line.strip()
        if line:
            data = json.loads(line)
            if data["id"] == docid:
                # print(data)
                print(len(data['title'].split()))
                break

In [ ]:
data

In [ ]:
for chunk_num in range(0, 90):
    print(f'sailctl job create bm25search{chunk_num}  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing; bash search_bm25.sh {chunk_num} " --debug')

In [ ]:
sailctl job create bm25search0  --args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing; bash search_bm25.sh 0 " --debug